# <font color="red">Solar Radiation Prediction with Support Vector Regressor with Poly Kernel Degree 2</font>

# Library Import

In [1]:
# Data manipulation and visualization
import pandas as pd
import math
import json

# Preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR

import sys
sys.path.append('../../') # Uncomment this line if running locally
# sys.path.append('/kaggle/input/weatherdata') # Uncomment this line if running on Kaggle
from historyManagement import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Datasets Import

In [2]:
# Load the weather dataset for visibility prediction
weather_data = pd.read_csv('../../FinalDatasets/finalDataset.csv') # Uncomment this line if running locally
# weather_data = pd.read_csv('/kaggle/input/weatherdata/finalDataset.csv') # Uncomment this line if running on Kaggle
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'])
weather_data.head()

,DateTime,Year,Month,Day,Hour,DaySegments,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,...,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy,Conditions,Icon
0,2023-01-01 00:00:00,2023,1,1,0,Late Night,0,0,0,1,...,0.49,1018.68,0.03,0.0,10.0,2.76,0.0,0.0,Clear,clear-night
1,2023-01-01 01:00:00,2023,1,1,1,Late Night,0,0,0,1,...,0.54,1018.03,0.11,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
2,2023-01-01 02:00:00,2023,1,1,2,Late Night,0,0,0,1,...,30.51,1017.56,0.03,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
3,2023-01-01 03:00:00,2023,1,1,3,Late Night,0,0,0,1,...,49.23,1018.05,0.00,0.0,10.0,2.28,0.0,0.0,Clear,clear-night
4,2023-01-01 04:00:00,2023,1,1,4,Late Night,0,0,0,1,...,49.90,1018.00,86.17,0.0,10.0,1.27,0.0,0.0,Partially cloudy,fog


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

In [4]:
weather_data.describe()

,DateTime,Year,Month,Day,Hour,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,DaySegments_Midday,...,Windgust,WindSpeed,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy
count,14616,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,...,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000
mean,2023-11-01 11:30:00,2023.400657,5.720854,15.735632,11.500000,0.131294,0.060550,0.098043,0.249726,0.116585,...,17.068560,8.090783,159.739178,1008.265012,53.579923,2.263555,16.504787,4.368134,227.640683,0.819217
min,2023-01-01 00:00:00,2023.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.660000,0.000000,0.000000,982.420000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,2023-06-02 05:45:00,2023.000000,3.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.170000,2.870000,80.965000,1003.277500,26.670000,0.000000,10.000000,3.990000,0.000000,0.000000
50%,2023-11-01 11:30:00,2023.000000,6.000000,16.000000,11.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14.645000,7.670000,163.790000,1007.970000,51.540000,0.000000,10.000000,4.650000,10.665000,0.010000
75%,2024-04-01 17:15:00,2024.000000,8.000000,23.000000,17.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.410000,11.750000,241.402500,1013.400000,87.790000,4.740000,12.580000,4.650000,459.745000,1.660000
max,2024-08-31 23:00:00,2024.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,84.120000,71.970000,360.000000,1022.060000,100.000000,10.000000,97.740000,24.100000,1026.650000,3.700000
std,NaN,0.490048,3.195073,8.803610,6.922423,0.337734,0.238511,0.297383,0.432869,0.320936,...,10.195911,6.250203,104.422027,6.010683,35.166637,3.083074,15.031827,1.376994,307.219387,1.106444


# Data Split

In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for train_idx, test_idx in sss.split(weather_data, weather_data['DaySegments']):
    weather_data_train = weather_data.iloc[train_idx]
    weather_data_test = weather_data.iloc[test_idx]

In [6]:
# Show the count of each unique class in DaySegments
class_counts = weather_data['DaySegments'].value_counts().reset_index()
class_counts.columns = ['DaySegments', 'Count']

print("Distribution of data based on DaySegments:")
print(class_counts)

Distribution of data based on DaySegments:
     DaySegments  Count
0     Late Night   3650
1        Morning   3504
2      Afternoon   1919
3         Midday   1704
4          Night   1521
5        Evening   1433
6  Early Morning    885


In [7]:
print("Distribution of DaySegments in Training Set:")
print(weather_data_train['DaySegments'].value_counts())

print("\nDistribution of DaySegments in Test Set:")
print(weather_data_test['DaySegments'].value_counts())

Distribution of DaySegments in Training Set:
DaySegments
Late Night       2920
Morning          2803
Afternoon        1535
Midday           1363
Night            1217
Evening          1146
Early Morning     708
Name: count, dtype: int64

Distribution of DaySegments in Test Set:
DaySegments
Late Night       730
Morning          701
Afternoon        384
Midday           341
Night            304
Evening          287
Early Morning    177
Name: count, dtype: int64


# Some Reused Parameters

In [8]:
model_name_for_saving = "SVR Poly 2"
target_name_for_saving = "SolarRadiation"
poly_degree = 2

# All Features

In [9]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_train = weather_data_train['SolarRadiation']

In [10]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_test = weather_data_test['SolarRadiation']

In [11]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [12]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


## MinMax Scaler

In [13]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [14]:
X_train

array([[0.72727273, 0.83333333, 0.43478261, ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.26666667, 0.69565217, ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.43333333, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.86666667, 0.43478261, ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.4       , 0.91304348, ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.26666667, 0.34782609, ..., 0.8925    , 0.29      ,
        0.27812962]], shape=(11692, 27))

In [15]:
X_test

array([[0.63636364, 0.06666667, 0.95652174, ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.26666667, 0.65217391, ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.03333333, 0.2173913 , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.2       , 0.73913043, ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.13333333, 0.86956522, ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.73333333, 0.95652174, ..., 0.5161    , 0.        ,
        0.07388643]], shape=(2924, 27))

### Default Parameters

In [16]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [17]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 55.5 s
Wall time: 56.3 s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [18]:
y_pred = training_model.predict(X_test)
y_pred

array([-4.46570762e-02,  8.82978738e+01, -1.24293005e-01, ...,
        2.11908278e+02,  9.90937995e-02,  6.08499105e-02], shape=(2924,))

In [19]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [20]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [21]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 311.4237067394816
MAE Score: 10.773828694055581
R2 Score: 0.9966883390711857
RMSE Score: 17.64720110214313


In [22]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 311.423707, R²: 0.996688
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Standard Scaler

In [23]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [24]:
X_train

array([[ 1.01884116,  1.17229173, -0.21938027, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461, -0.75947244,  0.64624572, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -0.1913065 , -1.66209024, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943,  1.28592492, -0.21938027, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264, -0.30493969,  1.36760071, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.75947244, -0.50792226, ...,  1.01722469,
         0.20430944,  0.85928021]], shape=(11692, 27))

In [25]:
X_test

array([[ 0.70637943, -1.44127155,  1.51187171, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598, -0.75947244,  0.50197472, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -1.55490474, -0.94073525, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092, -0.98673881,  0.79051672, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747, -1.21400518,  1.22332971, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  0.83139217,  1.51187171, ..., -0.05293696,
        -0.73592134, -0.43343649]], shape=(2924, 27))

### Default Parameters

In [26]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [27]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 2min 34s
Wall time: 2min 36s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [28]:
y_pred = training_model.predict(X_test)
y_pred

array([6.95572498e-03, 8.79420641e+01, 5.91498504e-02, ...,
       2.10111770e+02, 1.41853062e-01, 5.61194491e-02], shape=(2924,))

In [29]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [30]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [31]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 310.99305907141314
MAE Score: 10.723449126291646
R2 Score: 0.9966929185525341
RMSE Score: 17.634995295474653


In [32]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 310.993059, R²: 0.996693
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Robust Scaler

In [33]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [34]:
X_train

array([[ 0.6       ,  0.66666667, -0.16666667, ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        , -0.46666667,  0.33333333, ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -0.13333333, -1.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       ,  0.73333333, -0.16666667, ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        , -0.2       ,  0.75      , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.46666667, -0.33333333, ...,  0.61755563,
         0.59917355,  7.5       ]], shape=(11692, 27))

In [35]:
X_test

array([[ 4.00000000e-01, -8.66666667e-01,  8.33333333e-01, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.66666667e-01,  2.50000000e-01, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -9.33333333e-01, -5.83333333e-01, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01, -6.00000000e-01,  4.16666667e-01, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01, -7.33333333e-01,  6.66666667e-01, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  4.66666667e-01,  8.33333333e-01, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]],
      shape=(2924, 27))

### Default Parameters

In [36]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [37]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 26.7 s
Wall time: 27.3 s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [38]:
y_pred = training_model.predict(X_test)
y_pred

array([ 20.84760641,  98.3220352 ,   0.74302969, ..., 198.13703477,
        73.89982179,  28.68854648], shape=(2924,))

In [39]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [40]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [41]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 41430.08901031777
MAE Score: 131.1077899686584
R2 Score: 0.5594349303422121
RMSE Score: 203.54382577302061


In [42]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 41430.089010, R²: 0.559435
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


# Selected Features

In [43]:
numeric_cols = weather_data.select_dtypes(include=[float, int]).columns
correlations = weather_data[numeric_cols].corr()['SolarRadiation'].sort_values(ascending=False)
print(correlations)

SolarRadiation               1.000000
SolarEnergy                  0.999867
UVIndex                      0.998315
DaySegments_Midday           0.594891
Temp                         0.400378
FeelsLike                    0.344429
DaySegments_Morning          0.244659
WindSpeed                    0.216803
DaySegments_Afternoon        0.215785
WindDir                      0.180924
SevereRisk                   0.125829
Hour                         0.095598
Season_Summer                0.093764
Visibility                   0.054367
SeaLevelPressure             0.026622
Year                         0.021999
Dew                          0.013910
Day                         -0.001404
PrecipProb                  -0.013477
Season_Rainy                -0.033059
Season_Winter               -0.035395
Season_Autumn               -0.035413
Precip                      -0.039079
CloudCover                  -0.055418
Month                       -0.061339
Windgust                    -0.100270
DaySegments_

In [44]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

## Run Again

In [45]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Radiation
                                                'Dew', # low correlation with Solar Radiation
                                                'Precip', # low correlation with Solar Radiation
                                                'PrecipProb']) # low correlation with Solar Radiation
y_train = weather_data_train['SolarRadiation']

In [46]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Radiation
                                                'Dew', # low correlation with Solar Radiation
                                                'Precip', # low correlation with Solar Radiation
                                                'PrecipProb']) # low correlation with Solar Radiation
y_test = weather_data_test['SolarRadiation']

In [47]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [48]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


### MinMax Scaler

In [49]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [50]:
X_train

array([[0.72727273, 0.43478261, 0.        , ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.69565217, 1.        , ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.        , 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.43478261, 0.        , ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.91304348, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.34782609, 0.        , ..., 0.8925    , 0.29      ,
        0.27812962]], shape=(11692, 22))

In [51]:
X_test

array([[0.63636364, 0.95652174, 0.        , ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.65217391, 1.        , ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.2173913 , 0.        , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.73913043, 1.        , ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.86956522, 0.        , ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.95652174, 0.        , ..., 0.5161    , 0.        ,
        0.07388643]], shape=(2924, 22))

#### Default Parameters

In [52]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [53]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 32.2 s
Wall time: 32.5 s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [54]:
y_pred = training_model.predict(X_test)
y_pred

array([-1.66326746e-01,  9.58421512e+01, -1.31490683e-01, ...,
        2.10630426e+02,  2.25346425e-01,  7.01772575e-03], shape=(2924,))

In [55]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [56]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [57]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 307.86378405689055
MAE Score: 10.682856850774796
R2 Score: 0.9967261950744456
RMSE Score: 17.546047533757868


In [58]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 311.423707 → 307.863784 (lower is better)
   R²:  0.996688 → 0.996726 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


### Standard Scaler

In [59]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [60]:
X_train

array([[ 1.01884116, -0.21938027, -0.38875095, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461,  0.64624572,  2.57234098, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -1.66209024, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943, -0.21938027, -0.38875095, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264,  1.36760071, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.50792226, -0.38875095, ...,  1.01722469,
         0.20430944,  0.85928021]], shape=(11692, 22))

In [61]:
X_test

array([[ 0.70637943,  1.51187171, -0.38875095, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598,  0.50197472,  2.57234098, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -0.94073525, -0.38875095, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092,  0.79051672,  2.57234098, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747,  1.22332971, -0.38875095, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  1.51187171, -0.38875095, ..., -0.05293696,
        -0.73592134, -0.43343649]], shape=(2924, 22))

#### Default Parameters

In [62]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [63]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 1min 17s
Wall time: 1min 18s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [64]:
y_pred = training_model.predict(X_test)
y_pred

array([-7.86769807e-02,  9.67149327e+01, -3.64082770e-02, ...,
        2.09091103e+02,  3.31327328e-01,  6.82034271e-02], shape=(2924,))

In [65]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [66]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [67]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 307.6992332461421
MAE Score: 10.697371962297638
R2 Score: 0.9967279448978501
RMSE Score: 17.541357793686956


In [68]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 310.993059 → 307.699233 (lower is better)
   R²:  0.996693 → 0.996728 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


### Robust Scaler

In [69]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [70]:
X_train

array([[ 0.6       , -0.16666667,  0.        , ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        ,  0.33333333,  1.        , ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -1.        ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       , -0.16666667,  0.        , ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        ,  0.75      ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.33333333,  0.        , ...,  0.61755563,
         0.59917355,  7.5       ]], shape=(11692, 22))

In [71]:
X_test

array([[ 4.00000000e-01,  8.33333333e-01,  0.00000000e+00, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  1.00000000e+00, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -5.83333333e-01,  0.00000000e+00, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01,  4.16666667e-01,  1.00000000e+00, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01,  6.66666667e-01,  0.00000000e+00, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  8.33333333e-01,  0.00000000e+00, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]],
      shape=(2924, 22))

#### Default Parameters

In [72]:
training_model = SVR(kernel='poly', degree=poly_degree, tol=0.0001, C=100, epsilon=0.2, coef0=1.0)
training_model

,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [73]:
%%time
training_model.fit(X_train, y_train)

CPU times: total: 1min 35s
Wall time: 1min 36s


,kernel,'poly'
,degree,2
,gamma,'scale'
,coef0,1.0
,tol,0.0001
,C,100
,epsilon,0.2
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [74]:
y_pred = training_model.predict(X_test)
y_pred

array([-1.21949739e-01,  8.47152413e+01,  1.03066123e-01, ...,
        2.08958396e+02,  3.12571396e-01, -1.30624824e-01], shape=(2924,))

In [75]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [76]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [77]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 319.0817375002903
MAE Score: 10.98903066049519
R2 Score: 0.9966069040336039
RMSE Score: 17.862859163647077


In [78]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with better performance!
   MSE: 41430.089010 → 319.081738 (lower is better)
   R²:  0.559435 → 0.996607 (higher is better)
   Features: 27 → 22
💾 Registry updated: Model_Training_History/History_Regression.csv


# All Performance

In [79]:
show_model_history('regression', model_name=model_name_for_saving, target_column=target_name_for_saving)


📊 REGRESSION Model Performance History
     Model         Scaler         Target  Features        MSE       MAE      RMSE       R²
SVR Poly 2 StandardScaler SolarRadiation        22 307.699233 10.697372 17.541358 0.996728
SVR Poly 2   MinMaxScaler SolarRadiation        22 307.863784 10.682857 17.546048 0.996726
SVR Poly 2   RobustScaler SolarRadiation        22 319.081738 10.989031 17.862859 0.996607

📈 Total models shown: 3
🏆 Best R² Score: 0.996728 (SVR Poly 2 + StandardScaler for SolarRadiation)


# <center><font size="50" color="red">Thank You</font></center>